In [1]:
def read_data(path):
    f = open(path,"r")
    return f.readlines()[0].strip()

In [2]:
full_dataset = read_data("data.txt")
full_dataset

'420D5A802122FD25C8CD7CC010B00564D0E4B76C7D5A59C8C014E007325F116C958F2C7D31EB4EDF90A9803B2EB5340924CA002761803317E2B4793006E28C2286440087C5682312D0024B9EF464DF37EFA0CD031802FA00B4B7ED2D6BD2109485E3F3791FDEB3AF0D8802A899E49370012A926A9F8193801531C84F5F573004F803571006A2C46B8280008645C8B91924AD3753002E512400CC170038400A002BCD80A445002440082021DD807C0201C510066670035C00940125D803E170030400B7003C0018660034E6F1801201042575880A5004D9372A520E735C876FD2C3008274D24CDE614A68626D94804D4929693F003531006A1A47C85000084C4586B10D802F5977E88D2DD2898D6F17A614CC0109E9CE97D02D006EC00086C648591740010C8AF14E0E180253673400AA48D15E468A2000ADCCED1A174218D6C017DCFAA4EB2C8C5FA7F21D3F9152012F6C01797FF3B4AE38C32FFE7695C719A6AB5E25080250EE7BB7FEF72E13980553CE932EB26C72A2D26372D69759CC014F005E7E9F4E9FA7D3653FCC879803E200CC678470EC0010E82B11E34080330D211C663004F00101911791179296E7F869F9C017998EF11A1BCA52989F5EA778866008D8023255DFBB7BD2A552B65A98ECFEC51D540209DFF2FF2B9C1B9FE5D6A469F81590079160094CD73D85FD2699C5C9DCF21

In [3]:
hexa_mapping = {
    "0": "0000",
    "1": "0001",
    "2": "0010",
    "3": "0011",
    "4": "0100",
    "5": "0101",
    "6": "0110",
    "7": "0111",
    "8": "1000",
    "9": "1001",
    "A": "1010",
    "B": "1011",
    "C": "1100",
    "D": "1101",
    "E": "1110",
    "F": "1111",
}

# Part 1

In [4]:
def hexa_to_bits(hexa: str) -> str:
    return "".join([hexa_mapping[i] for i in hexa])

In [5]:
assert hexa_to_bits("D2FE28") == '110100101111111000101000'

In [6]:
def convert_binary(string: str):
    exp = len(string) - 1
    value = 0
    for i in string:
        value += int(i) * 2**exp
        exp -= 1
    return value

In [7]:
def get_single_version(binary: str) -> int:
    return convert_binary(binary[0:3])

In [8]:
assert get_single_version("110100101111111000101000") == 6

In [9]:
def get_type(binary: str) -> int:
    return convert_binary(binary[3:6])

In [10]:
assert get_type("110100101111111000101000") == 4

In [11]:
def get_length_type(binary: str) -> int:
    return binary[6]

In [12]:
def get_literal_end(binary: str) -> int:
    index = 6
    while binary[index] == "1":
        index += 5
    return index + 4

In [ ]:
# TODO: Introduce a node class to better handle this

In [13]:
def get_size(value):
    if isinstance(value, str):
        return len(value)
    else:
        return len(list(value.keys())[0])

In [14]:
def parse_one(binary: str) -> str:
    if get_type(binary) == 4:
        end = int(get_literal_end(binary))
        return binary[0: end+1]
    else:
        children = []
        length_type = get_length_type(binary)
        if length_type == "1":
            count = convert_binary(binary[7:7+11])
            children_binary = binary[7+11:]
            full_size = 7+11
            while len(children) < count:
                new_binary = parse_one(children_binary)
                children.append(new_binary)
                new_size = get_size(new_binary)
                full_size += new_size
                children_binary = children_binary[new_size:]
            return {binary[:full_size]: children}
        else:
            size = convert_binary(binary[7:7+15])
            children_binary = binary[7+15:7+15+size]
            while len(children_binary)>6:
                new_binary = parse_one(children_binary)
                children.append(new_binary)
                children_binary = children_binary[get_size(new_binary):]
            return {binary[:7+15+size]: children}

In [15]:
def parse(hexa: str) -> int:
    binary = hexa_to_bits(hexa)
    parsed = parse_one(binary)
    return parsed

In [16]:
def get_sum_rec(data) -> int:
    if isinstance(data, str):
        return get_single_version(data)
    else:
        return sum([get_single_version(key) + sum([get_sum_rec(i) for i in values]) for key, values in data.items() ])

In [17]:
def get_version_sum(hexa: str):
    parsed = parse(hexa)
    return get_sum_rec(parsed)

In [18]:
assert get_version_sum("8A004A801A8002F478") == 16
assert get_version_sum("620080001611562C8802118E34") == 12
assert get_version_sum("C0015000016115A2E0802F182340") == 23
assert get_version_sum("A0016C880162017C3686B18A3D4780") == 31

In [19]:
get_version_sum(full_dataset)

993

# Part 2

In [20]:
def get_literal_number(literal):
    index = 7
    bits = ""
    while index < len(literal):
        bits += literal[index: index + 4]
        index += 5
    return convert_binary(bits)

In [21]:
assert get_literal_number("110100101111111000101") == 2021

In [22]:
get_literal_number("1101001011111110001010")

2021

In [23]:
def prod(data):
    if len(data) == 1:
        return data[0]
    else:
        return data[0] * prod(data[1:])

In [24]:
def compute_score_rec(data):
    if isinstance(data, str):
        return get_literal_number(data)
    else:
        operator = get_type(list(data.keys())[0])
        children = [compute_score_rec(v) for v in list(data.values())[0]]
        if operator == 0:
            return sum(children)
        if operator == 1:
            return prod(children)
        if operator == 2:
            return min(children)
        if operator == 3:
            return max(children)
        if operator == 5:
            return 1 if children[0] > children[1] else 0
        if operator == 6:
            return 1 if children[0] < children[1] else 0
        if operator == 7:
            return 1 if children[0] == children[1] else 0
        raise Exception("unknown operator: " + str(operator))

In [25]:
def get_score(hexa: str) -> int:
    parsed = parse(hexa)
    return compute_score_rec(parsed)

In [26]:
assert get_score("C200B40A82") == 3
assert get_score("04005AC33890") == 54
assert get_score("880086C3E88112") == 7
assert get_score("CE00C43D881120") == 9
assert get_score("D8005AC2A8F0") == 1
assert get_score("F600BC2D8F") == 0
assert get_score("9C005AC2F8F0") == 0
assert get_score("9C0141080250320F1802104A08") == 1

In [27]:
get_score(full_dataset)

144595909277